In [24]:
import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import csv
from dragon import dragonV
import os

# data_parsing

In [25]:
root_path = 'C:/Users/admin/Desktop/Philadelphia/real_final_data/'
subject_list = ['pp01', 'pp02', 'pp009', 'pp085', 'pp086', 'pp087', 'pp088', 'pp089'] 
#front, rear 2가지 존재. 파일명 format : {gait_catergory}_{front || rear}.xlsx
gait_category_list = ['gait1', 'gait2', 'fast', 'preferred', 'reaction', 'slow', 'stroop', 'turn']
seleted_openpose_joint_idx_list = [8, 9, 10, 11, 12, 13, 14, 19, 20, 21, 22, 23, 24]

## xlsx -> nomalize -> lower joint -> frame & gt pair

In [28]:
all_subject_data = []
for each_subject in subject_list:
    #root/ppxx/
    each_subject_path = root_path + each_subject + '/'
    files = os.listdir(each_subject_path)

    ###xlsx_list###
    excel_name_list = [file for file in files if file.endswith('.xlsx')]
    
    ###gt_list####
    tmp = os.listdir(each_subject_path + 'gt/')
    gt_name_list = [file for file in tmp if file.endswith('.npy')]

    #ppxx/{each_excel_name}
    #하나의 엑셀 파일에 대한 반복문
    for each_excel in excel_name_list:
        ##xlsx -> list##
        frame_data_list = dragonV.xlsx2data(each_subject_path + each_excel)
        #print(f'{each_excel} : {len(frame_data_list[0])}')
        ##list -> normalize##
        norm_frame_data_list = dragonV.nomalize_data(frame_data_list)
        #print(f'{each_subject}->{each_excel} : {len(norm_frame_data_list[0])}')

        ##noramlize -> select lower joint pos##
        selected_norm_frame_data_list = dragonV.get_selected_joint_pos_frame_list(norm_frame_data_list, seleted_openpose_joint_idx_list)
        #print(f'{each_subject}->{each_excel} : {len(selected_norm_frame_data_list[0])}')
        
        ## 여기까진 왔음...
        ##frame data : gt pair##
        for each_gt in gt_name_list:
            #확장자명 제거
            each_gt_name = each_gt[:-4]
            if each_gt_name in each_excel:
                #print(f'{each_subject}:{each_excel} -> {each_gt_name}')
                right_gt_np = np.load(each_subject_path + '/gt/' + each_gt)
                right_gt_list = right_gt_np.tolist()

                data_gt_pair_list = dragonV.make_dataAndGtPair(selected_norm_frame_data_list, right_gt_list)
                all_subject_data.append(data_gt_pair_list)
            else:
                continue


In [35]:
d1 = len(all_subject_data)
d2 = len(all_subject_data[0])
d3 = len(all_subject_data[0][0])
d4_0 = len(all_subject_data[0][0][0])
d4_1 = len(all_subject_data[0][0][1])


In [33]:
print('video_num :',d1)
print('frame_num :',d2)
print('features || label :',d3)
print('d4_0 :',d4_0)
print('d4_1 :',d4_1)

video_num : 73
frame_num : 253
features || label : 2
d4_0 : 26
d4_1 : 4


## list -> torch

In [6]:
video_num = 0
frame_num = 0
features_labels = 0
np.array(all_subject_data[video_num][frame_num][features_labels])

array([0.98548271, 0.92060491, 0.98759102, 0.96146204, 0.9881226 ,
       0.9740866 , 0.99484523, 0.98557527, 0.81086829, 0.91704873,
       0.70520791, 0.97594994, 0.71989963, 0.9997064 , 0.62189694,
       0.89097924, 0.9188043 , 0.9825343 , 0.73717281, 0.98529396,
       0.99225403, 0.97184447, 0.98237504, 0.9717276 , 0.99244709,
       0.97882186])

[
    [
        [features],
        [features],
        ......... ,
        [features]
    ],
    [
        [features],
        [features],
        ......... ,
        [features]
    ],
    [
        [features],
        [features],
        ......... ,
        [features]
    ],    
]

In [27]:
print(len(all_subject_data))
print(len(all_subject_data[0]))
print(len(all_subject_data[0][0][0]))

73
163
26


In [6]:
# timesteps이 9라서 앞, 뒤로 레이블 데이터 4개씩 버려야 함.
'''
each_video : frame_num x (features || label) x (26 || 4)
'''
trimmed_data = []
trimmed_label = []

for each_video in all_subject_data:
    #각 비디오의 전체 프레임 길이
    video_len = len(each_video)
    #frame_number x each_element(26)
    total_frame_features_list = []
    #frame_number x each_label(4)
    total_frame_label_list = []
    
    #각각의 비디오에서 frame_data와 frame별 label 값 추출
    for i in range(0, video_len):
        total_frame_features_list.append(each_video[i][0])
        total_frame_label_list.append(each_video[i][1])

    #timesteps 수만큼 frame data 묶기.
    for idx in range(0, len(total_frame_features_list) - 8):
        trimmed_data.append(total_frame_features_list[idx:idx+9])
    
    del total_frame_label_list[0:4]
    del total_frame_label_list[-4:]
    
    for label in total_frame_label_list:
        trimmed_label.append(label)
        

In [7]:
####data info###
print(f'sample_num : {len(trimmed_data)}')
print(f'timesteps : {len(trimmed_data[0])}')
print(f'input_size : {len(trimmed_data[0][0])}')
##########

print('\n')
###label###
print(f'sample_num : {len(trimmed_label)}')
print(f'label_size : {len(trimmed_label[0])}')

########

sample_num : 45834
timesteps : 9
input_size : 26


sample_num : 45834
label_size : 4


In [93]:
'''
# data 저장된 곳
trimmed_data : list
trimmed_label : list

trimmed_data : sample_size x timesteps(9) x input_size(26)
trimmed_label : sample_size x label_size(4)
'''

'\n# data 저장된 곳\ntrimmed_data : list\ntrimmed_label : list\n\ntrimmed_data : sample_size x timesteps(9) x input_size(26)\ntrimmed_label : sample_size x label_size(4)\n'

# data_preprocessing

outputs, _status = cell(output)<br>

(모든 timesteps에 대한 결과)<br>
outputs -> [bactch x timesteps x output]<br><br>
(마지막 timestpes에 대한 결과)<br>
_status -> [1 x batch x output]<br>

# data to tensor

In [8]:
input_data_np = np.array(trimmed_data)
label_data_np = np.array(trimmed_label)

In [9]:
print(len(trimmed_data))
print(len(trimmed_data[0]))

45834
9


In [36]:
input_data_np = np.load('C:/Users/admin/Desktop/Philadelphia/input_data.npy')
label_data_np = np.load('C:/Users/admin/Desktop/Philadelphia/label_data.npy')

In [37]:
input_data_tensor = torch.tensor(input_data_np, dtype=torch.float32)
label_tensor = torch.tensor(label_data_np, dtype=torch.long)

In [38]:
has_nan = torch.isnan(input_data_tensor).any().item()
print("NaN 값이 있는지 여부:", has_nan)

NaN 값이 있는지 여부: False


In [26]:
# NaN 값을 포함한 인덱스 확인
nan_indices = torch.isnan(input_data_tensor).nonzero()
print("NaN 값이 있는 인덱스:", nan_indices)

NaN 값이 있는 인덱스: tensor([[29822,     8,     0],
        [29822,     8,     1],
        [29822,     8,     2],
        ...,
        [31557,     0,    23],
        [31557,     0,    24],
        [31557,     0,    25]])


In [27]:
# NaN 값을 평균 값으로 대체하는 함수
def replace_nan_with_mean(tensor):
    # NaN 값을 평균 값으로 대체하기 위해 평균 값을 계산
    mean_values = torch.nanmean(tensor, dim=(0, 1, 2))
    # NaN 값을 대체할 인덱스를 찾음
    nan_indices = torch.isnan(tensor)
    # NaN 값을 평균 값으로 대체
    tensor[nan_indices] = mean_values
    return tensor

In [28]:
input_data_tensor = replace_nan_with_mean(input_data_tensor)

In [39]:
print(input_data_tensor.shape)
print(label_tensor.shape)

torch.Size([45834, 9, 26])
torch.Size([45834, 4])


# hyper-parameter

In [45]:
input_size = 26
output_size = 4
hidden_size = 128
batch_size = 10
num_epochs = 100
learning_rate = 0.001

In [41]:
from torch.utils.data import TensorDataset, DataLoader, Dataset

In [42]:
dataset = TensorDataset(input_data_tensor, label_tensor)
data_loader = DataLoader(dataset, batch_size=batch_size, shuffle=True, drop_last=True)

In [43]:
#DataLoader 테스트
for batch_idx, (data, target) in enumerate(data_loader):
    print("Batch", batch_idx)
    print("Data shape:", data.dtype)  # 미니배치의 입력 데이터 모양
    print("Target shape:", target.dtype)

    if batch_idx == 0:
        break

Batch 0
Data shape: torch.float32
Target shape: torch.int64


## network 정의

In [44]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(LSTM, self).__init__()
        
        self.hidden_size = hidden_size
        
        self.lstm = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.fc = nn.Linear(hidden_size, output_size)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        _, (hidden, _) = self.lstm(x)
        out = self.fc(hidden[-1])
        out = self.sigmoid(out)
        return out

## criterian && optimizer 정의

In [46]:
model = LSTM(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

## training network

In [47]:
for batch_idx, (data, target) in enumerate(data_loader):

    print(data.shape)
    outputs = model(data)
    print(outputs)
    print(target)
    loss = criterion(outputs, target.float())
    if batch_idx == 0:
        break
    

torch.Size([10, 9, 26])
tensor([[0.4951, 0.5164, 0.4725, 0.5228],
        [0.4976, 0.5153, 0.4725, 0.5212],
        [0.4948, 0.5170, 0.4718, 0.5238],
        [0.4971, 0.5152, 0.4737, 0.5208],
        [0.4920, 0.5171, 0.4745, 0.5208],
        [0.5041, 0.5109, 0.4808, 0.5107],
        [0.4955, 0.5164, 0.4730, 0.5223],
        [0.4943, 0.5172, 0.4712, 0.5242],
        [0.4955, 0.5164, 0.4724, 0.5224],
        [0.4968, 0.5154, 0.4725, 0.5223]], grad_fn=<SigmoidBackward0>)
tensor([[0, 0, 1, 1],
        [0, 1, 0, 0],
        [1, 1, 1, 1],
        [0, 0, 0, 0],
        [0, 0, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 0, 0],
        [1, 1, 1, 1]])


In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)

cuda


In [49]:
def train_network(model, data_loader, num_epochs, criterion, optimizer):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    for epoch in range(1, num_epochs + 1):
        model.train()

        running_loss = 0.0
        correct_predictions = 0
        total_predictions = 0

        for inputs, labels in data_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            optimizer.zero_grad() 

            outputs = model(inputs)
            #print(outputs)
            loss = criterion(outputs, labels.float())
            loss.backward()
            optimizer.step()

            running_loss += loss.item()

            # 예측값 계산
            predicted = (outputs > 0.5).float()
            total_predictions += labels.size(0)
            correct_predictions += (predicted == labels).sum().item()

        epoch_loss = running_loss / len(data_loader.dataset)
        if(epoch % 10 == 0):
            print(f'Epoch [{epoch}/{num_epochs}], Loss: {epoch_loss:.4f}')


## train_space

In [50]:
train_network(model, data_loader, num_epochs=num_epochs, criterion=criterion, optimizer=optimizer)

Epoch [10/1000], Loss: 0.0560
Epoch [20/1000], Loss: 0.0485
Epoch [30/1000], Loss: 0.0434
Epoch [40/1000], Loss: 0.0402
Epoch [50/1000], Loss: 0.0376
Epoch [60/1000], Loss: 0.0355
Epoch [70/1000], Loss: 0.0333
Epoch [80/1000], Loss: 0.0316
Epoch [90/1000], Loss: 0.0300
Epoch [100/1000], Loss: 0.0290
Epoch [110/1000], Loss: 0.0278
Epoch [120/1000], Loss: 0.0271
Epoch [130/1000], Loss: 0.0262
Epoch [140/1000], Loss: 0.0255
Epoch [150/1000], Loss: 0.0248
Epoch [160/1000], Loss: 0.0249
Epoch [170/1000], Loss: 0.0236
Epoch [180/1000], Loss: 0.0230
Epoch [190/1000], Loss: 0.0226
Epoch [200/1000], Loss: 0.0223
Epoch [210/1000], Loss: 0.0219
Epoch [220/1000], Loss: 0.0215
Epoch [230/1000], Loss: 0.0208
Epoch [240/1000], Loss: 0.0209
Epoch [250/1000], Loss: 0.0202
Epoch [260/1000], Loss: 0.0204
Epoch [270/1000], Loss: 0.0200
Epoch [280/1000], Loss: 0.0198
Epoch [290/1000], Loss: 0.0193
Epoch [300/1000], Loss: 0.0192
Epoch [310/1000], Loss: 0.0189
Epoch [320/1000], Loss: 0.0189
Epoch [330/1000],

In [51]:
torch.save(model.state_dict(), '../Philadelphia/models/1000_epochs.pth')

In [53]:
model_100 = LSTM(input_size, hidden_size, output_size)
criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
train_network(model_100, data_loader, num_epochs=100, criterion=criterion, optimizer=optimizer)

Epoch [10/100], Loss: 0.0696
Epoch [20/100], Loss: 0.0696
Epoch [30/100], Loss: 0.0696
Epoch [40/100], Loss: 0.0696
Epoch [50/100], Loss: 0.0696
Epoch [60/100], Loss: 0.0696
Epoch [70/100], Loss: 0.0696
Epoch [80/100], Loss: 0.0696
Epoch [90/100], Loss: 0.0696
Epoch [100/100], Loss: 0.0696


In [54]:
torch.save(model.state_dict(), '../Philadelphia/models/100_epochs.pth')

In [ ]:
def evaluate(model, test_loader, criterion, threshold):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.eval()

    accuracy = 0.0
    precision = 0.0
    recall = 0.0
    f1 = 0.0

    test_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)

            outputs = model(inputs)
            


    # 다중 레이블 분류 평가 지표 계산, macro : 각 클래스에 대해 개별적으로 평가
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='macro')
    recall = recall_score(true_labels, predictions, average='macro')
    f1 = f1_score(true_labels, predictions, average='macro')

    print(f'Average Test Loss: {avg_loss:.4f}')
    print(f'Accuracy: {accuracy:.4f}, Precision: {precision:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}')